#INSURANCE CLAIM NOTEBOOK VERSION 2 

<a id="load_package"/><br/>
# INTRODUCTION

Special shoutout to:
https://github.com/osinkolu/Umoja-Hack-Africa-2022/blob/main/2nd_Place_Intermediate_Umoja2022_Intermediate.ipynb

This notebook has helped me realise what i was doing wrong. I merged the policies data without grouping them. The policy_number in policies had duplicates and as i was merging in my first notebook i lost a lot of useful information since i dropped the duplicates 

 


<a id="load_package"/><br/>
# LOADING NECESSARY LIBRARIES

In [ ]:
!pip install statsmodels

!pip install catboost

In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 2200
pd.options.display.max_columns = 2200
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.offline as pyo
import statsmodels.api as sm
import ipywidgets as widgets
import pylab
import warnings
import os
import sys

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split



from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

from scipy import stats
from IPython.display import display

warnings.filterwarnings('ignore')
pyo.init_notebook_mode()

<a id="load_df"/><br/>
# HANDLING THE DATASETS

In [ ]:
#Start by connecting gdrive into the google colab
from google.colab import drive
drive.mount('/content/gdrive')
path = "/content/gdrive/MyDrive/Insurane_claim"


In [ ]:
#import df
train = pd.read_csv(path+'/train.csv')
test=  pd.read_csv(path+'/test.csv')
policies = pd.read_csv(path+'/policies.csv')
submission = pd.read_csv(path+'/SampleSubmission.csv')

In [ ]:
train.shape,policies.shape,test.shape

In [ ]:
train.head()

In [ ]:
policies.head()

In [ ]:
policies.isnull().sum()

In [ ]:
policies['class_of_business'] = policies['class_of_business'].fillna('cob00031')

In [ ]:
policies['policy_number'].duplicated().sum()

In [ ]:
policies.shape

In [ ]:
#Find the number of days between  policies["to_date"] and policies['from_date']
policies['diff_days'] = (pd.to_datetime(policies["to_date"]) - pd.to_datetime(policies['from_date'])).dt.days

In [ ]:
pols = policies.groupby('policy_number').agg(
             prd_min = ('product_code', 'min'),
             prd_max = ('product_code', 'max'),
             prd_count = ('product_code', 'count'),
             ag_min = ('agent', 'min'),
             diff_days_min = ("diff_days", "min"),
             diff_days_max = ("diff_days", "max"),
             diff_days_mean = ("diff_days", "mean"),
             ag_max = ('agent', 'max'),
             class_min = ('class_of_business', 'min'),
             class_max = ('class_of_business', 'max'),).reset_index()

In [ ]:
pols.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
policy_cat = ["prd_max","ag_min", "prd_min","ag_max","class_min","class_max"]
for col in policy_cat:
  lr = LabelEncoder()
  pols[col] = lr.fit_transform(pols[col])


In [ ]:
pols.head()

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
df = pd.concat((train, test)).reset_index(drop=True)

In [ ]:
df.info()

In [ ]:
#check for missing values
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

In [ ]:
FEATURES=['claim_id', 'policy_number', 'claim_number', 'sum_insured', 'product',
       'agent', 'class_of_business', 'risk_type', 'loss_date', 'client_type',
       'renewal_frequency', 'primary_cause', 'secondary_cause', 'branch']

In [ ]:
df['policy_number'] = df['policy_number'].fillna(df['policy_number'].mode()[0])

In [ ]:
df.isnull().sum()

In [ ]:
df.apply(lambda x: len(x.unique()))

In [ ]:
merged_df = pd.merge(df,pols,on='policy_number',how ='left')
merged_df.shape,df.shape

In [ ]:
merged_df['claim_id'].duplicated().sum()

In [ ]:
merged_df.isnull().sum()

<a id="load_package"/><br/>
# FEATURE ENGINEERING

In [ ]:
date_columns = ['loss_date']
for c in date_columns:
    df[c] = pd.to_datetime(df[c])
    # extract new ceatures crom date
    df[c + '_day'] = df[c].dt.day
    
    df[c + '_year'] = df[c].dt.year
    df[c + '_month'] = df[c].dt.month
    df[c + '_weekday'] = df[c].dt.weekday
    df[c + '_quarter'] = df[c].dt.quarter


    # remove original df column
    df = df.drop(c, axis=1)

In [ ]:
# Separate train and test df from the combined DataFrame
train= df[:ntrain]
test = df[ntrain:]

# Check the shapes of the split dfset
train.shape, test.shape

In [ ]:
df['agent'].value_counts().plot(kind='bar')

In [ ]:
def agent_reduce(x):
    

    ag00037 = [ 'ag00037']
    ag00226 = ['ag00226']
    ag00068 = ['ag00068']
    ag00064 = [ 'ag00064','ag00021','ag00044','ag00083','ag00080']


    if x in ag00037:
        return 1
    elif x in ag00226:
        return 2
    elif x in ag00068:
        return 3
    elif x in ag00064:
        return 4


df['agent_reduced'] = df['agent'].apply(agent_reduce)

In [ ]:
df['class_of_business'].value_counts().plot(kind='bar')

In [ ]:
def business_reduce(x):
    

    ag00037 = [ 'cob00034']
    ag00226 = ['cob00009']
    ag00068 = ['cob00031','cob00010','cob00022','cob00013','cob00016','cob00023','cob00004','cob00015','cob00026','cob00001','cob00036','cob00003','cob00033','cob00027','cob00012','cob00002','cob00021','cob00029','cob00028','cob00024','cob00035','cob00007','cob00011','cob00030','cob00018','cob00017','cob00038','cob00008','cob00005','cob00006','cob00037','cob00019','cob00014','cob00025','cob00020',]



    if x in ag00037:
        return 1
    elif x in ag00226:
        return 2
    elif x in ag00068:
        return 3


df['cob_reduced'] = df['class_of_business'].apply(business_reduce)

In [ ]:
df['risk_type'].value_counts().plot(kind='bar')

In [ ]:
def risk_reduce(x):
    

    ag00037 = [ 'rt00023']
    ag00226 = ['rt00032']
    ag00068 = ['rt00006',"rt00033","rt00025","rt00024","rt00050","rt00002","rt00015","rt00013","rt00018","rt00001","rt00027","rt00026","rt00041","rt00039","rt00031","rt00012","rt00040","rt00022","rt00010","rt00043",
"rt00008","rt00011","rt00005","rt00009","rt00030","rt00003","rt00038","rt00028","rt00034","rt00037","rt00042","rt00035","rt00004","rt00019","rt00029","rt00044","rt00014","rt00017","rt00007","rt00020","rt00049","rt00036","rt00045","rt00016","rt00046","rt00047",
"rt00021","rt00048"]



    if x in ag00037:
        return 1
    elif x in ag00226:
        return 2
    elif x in ag00068:
        return 3



df['risk_reduced'] = df['risk_type'].apply(risk_reduce)





In [ ]:
df['client_type'].value_counts().plot(kind='bar')

In [ ]:
def client_int(x):
    

    ag00037 = [ 'ct0001']
    ag00226 = ['ct0003','ct0002']
    ag00068 = []


    if x in ag00037:
        return 1
    elif x in ag00226:
        return 2


df['client_int'] = df['client_type'].apply(client_int)

In [ ]:
df['renewal_frequency'].value_counts().plot(kind='bar')

In [ ]:
def renewal_reduce(x):
    

    ag00037 = [ 'rf0002']
    ag00226 = ['rf0001']
    ag00068 = ['rf0003','rf0005','rf0004']



    if x in ag00037:
        return 1
    elif x in ag00226:
        return 2
    elif x in ag00068:
        return 3
 


df['renewal_reduced'] = df['renewal_frequency'].apply(renewal_reduce)

In [ ]:
df['primary_cause'].value_counts().plot(kind='bar')

In [ ]:
def primary_reduce(x):
    

    ag00037 = [ 'pc0007']
    ag00226 = ["pc0002","pc0001","pc0012","pc0015","pc0017","pc0011","pc0004","pc0005","pc0003","pc0013","pc0009","pc0014","pc0018","pc0010","pc0019","pc0006","pc0008","pc0021","pc0016","pc0020",]



    if x in ag00037:
        return 1
    elif x in ag00226:
        return 2



df['primary_reduced'] = df['primary_cause'].apply(primary_reduce)


In [ ]:
df['secondary_cause'].value_counts().plot(kind='bar')


In [ ]:
def secondary_reduce(x):
    

    ag00037 = [ 'sc00023']
    ag00226 = ['sc00021','sc00024']
    ag00068 = ["sc00022","sc00004","sc00003","sc00029","sc00020","sc00025","sc00031","sc00032","sc00008","sc00050","sc00045","sc00048","sc00015","sc00053","sc00049","sc00051","sc00019","sc00007","sc00026","sc00061","sc00017","sc00059","sc00074","sc00030","sc00034","sc00002","sc00013","sc00009","sc00037","sc00018","sc00014","sc00038","sc00012","sc00005","sc00055","sc00039","sc00044","sc00057","sc00040",
"sc00066","sc00016","sc00064","sc00006","sc00075","sc00054","sc00011","sc00033","sc00056","sc00042","sc00067","sc00063","sc00043","sc00010","sc00036","sc00052","sc00041","sc00035","sc00027","sc00028","sc00058","sc00060","sc00046","sc00065","sc00076","sc00079","sc00068","sc00072","sc00077","sc00071","sc00062","sc00070","sc00047","sc00069",]
    


    if x in ag00037:
        return 1
    elif x in ag00226:
        return 2
    elif x in ag00068:
        return 3



df['secondary_reduced'] = df['secondary_cause'].apply(secondary_reduce)







In [ ]:
df['branch'].value_counts().plot(kind='bar')


In [ ]:
def branch_reduce(x):
    

    ag00037 = [ 'br00002']
    ag00226 = ['br00001']
    ag00068 = ["br00007","br00005","br00006","br00009","br00008","br00004","br00011","br00003","br00010",]

    


    if x in ag00037:
        return 1
    elif x in ag00226:
        return 2
    elif x in ag00068:
        return 3



df['branch_reduced'] = df['branch'].apply(branch_reduce)

In [ ]:
df['product'].value_counts().plot(kind='bar')

In [ ]:



def product_reduce(x):
    

    ag00037 = [ "prod00015"]
    ag00226 = ["prod00005","prod00027","prod00028","prod00002","prod00008","prod00012","prod00021","prod00011","prod00018","prod00029","prod00034","prod00031","prod00023","prod00032","prod00001","prod00013","prod00009","prod00030","prod00016","prod00036","prod00025","prod00007","prod00017","prod00022","prod00019","prod00006","prod00033","prod00003","prod00026","prod00014","prod00037","prod00004","prod00024","prod00035","prod00010","prod00020",]


    


    if x in ag00037:
        return 1
    elif x in ag00226:
        return 2



df['product_reduced'] = df['product'].apply(product_reduce)

<a id="load_package"/><br/>
# OUTLIER DETECTION AND HANDLING

In [ ]:


#finding outliers and skewness
plt.figure(figsize=(13,7))
plt.subplot(2,1,2)
sns.boxplot(df['target'])
plt.title("boxplot of Target")
plt.show()
plt.subplot(2,1,2)
sns.distplot(df["target"])
plt.show()
plt.figure(figsize=(13,7))
plt.subplot(2,1,2)
sns.boxplot(df['sum_insured'])
plt.title("boxplot of sum_insured")
plt.show()
plt.subplot(2,1,2)
sns.distplot(df["sum_insured"])
plt.title("distplot of sum_insured")
plt.show()
plt.figure(figsize=(13,7))
plt.subplot(2,1,2)
sns.boxplot(df['loss_date_year'])
plt.title("boxplot of to_date_year")
plt.show()
plt.subplot(2,1,2)
sns.distplot(df["loss_date_day"])
plt.title("distplot of to_date_day")
plt.show()
plt.figure(figsize=(13,7))
plt.subplot(2,1,2)
sns.boxplot(df['loss_date_month'])
plt.title("boxplot of from_date_month")
plt.show()


In [ ]:
df[['target','sum_insured']].describe()


In [ ]:
outliers = ['target','sum_insured']

df[outliers].describe(percentiles=[0.10,0.90])

In [ ]:
df['target'].loc[df['target'] > 7.422626e+04]  = 7.422626e+04	
df['sum_insured'].loc[df['sum_insured'] > 5.753700e+06]  = 5.753700e+06




In [ ]:
sns.boxplot(df['target'])

In [ ]:
sns.boxplot(df['sum_insured'])

<a id="load_package"/><br/>
#MORE FEATURE ENGINEERING

In [ ]:
df["causes"] = df["secondary_reduced"] + df["primary_reduced"]
df["risk_business"] = df["risk_reduced"] * df["cob_reduced"]
df['risk_product'] = df['risk_reduced'] * df['product_reduced']
df['risk_business_product'] = df['risk_reduced'] + df['cob_reduced'] + df['product_reduced']

df["branch_agency"] =  df["branch_reduced"]*df["agent_reduced"]
df['client_risk'] = df['client_int']*df['risk_reduced']
df['client_business'] = df['client_int'] + df['cob_reduced']
df['client_business_risk'] = df['client_int'] + df['cob_reduced'] + df['risk_reduced']
df['product_client_business_risk'] = df['product_reduced'] + df['client_business_risk']

df['renewals_client_business_risk'] = df['renewal_reduced'] * df['client_business_risk']

df['sum_insured_cause'] = df['sum_insured'] / df['causes']
df['sum_insured_product'] = df['sum_insured'] / df['product_reduced']
df['sum_insured_client'] = df['sum_insured'] / df['client_int']
df['sum_insured_risk'] = df['sum_insured'] / df['risk_reduced']
df['sum_insured_business'] = df['sum_insured'] / df['cob_reduced']
df['sum_insured_agent'] = df['sum_insured'] / df['agent_reduced']
df['sum_insured_branch'] = df['sum_insured'] / df['branch_reduced']
df['sum_insured_renewals'] = df['sum_insured'] / df['renewal_reduced']

df['sum_insured_year'] = df['sum_insured'] / df['loss_date_year']
df['sum_insured_year'] = df['sum_insured'] / df['loss_date_month']
df['sum_insured_year'] = df['sum_insured'] / df['loss_date_quarter']
df['sum_insured_year'] = df['sum_insured'] / df['loss_date_weekday']
df['sum_insured_year'] = df['sum_insured'] / df['loss_date_day']

df['renewal_product'] = df['renewal_reduced'] * df['product_reduced']
df['renewal_business'] = df['renewal_reduced'] * df['cob_reduced']
df['renewal_risk'] = df['renewal_reduced'] * df['risk_reduced']
df['renewal_type'] = df['renewal_reduced'] * df['client_int']

df['renewal_type'] = df['renewal_reduced'] * df['branch_agency']

df['full_loss_date'] = df['loss_date_day']+df['loss_date_month']+df['loss_date_year']
df['loss_month_year'] = df['loss_date_month'] + df['loss_date_year']





In [ ]:
object_df=df.select_dtypes(include=[object])
object_df.columns

In [ ]:
loss_date = df.groupby('loss_date_year').mean()
# loss_date.drop(['target','claim_id', 'policies_number', 'claim_number', 'product', 'agent',
#        'class_of_business', 'risk_type', 'client_type', 'renewal_frequency',
#        'primary_cause', 'secondary_cause', 'branch'], 1, inplace = True)
cols = []
for i in loss_date.columns:
    if i != 'loss_date_year':
        loss_date[i+'_loss_date_mean_all'] = loss_date[i]
        loss_date.drop(i, 1, inplace = True)
        cols.append(i+'_ten_mean_all')

df = pd.merge(df, loss_date, on='loss_date_year', how = 'left')


In [ ]:
df.head()

In [ ]:
object_cols = ['policy_number', 'claim_number', 'product', 'agent',
       'class_of_business', 'risk_type', 'client_type', 'renewal_frequency',
       'primary_cause', 'secondary_cause', 'branch']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[object_cols] = df[object_cols].apply(le.fit_transform)

In [ ]:
freq_columns = ["sum_insured","secondary_reduced","cob_reduced","renewal_reduced","risk_reduced","product_reduced","primary_reduced","agent_reduced","branch_reduced",]
for col in freq_columns:
   df[col+'_freq'] = df[col].map(df[col].value_counts().to_dict())/len(df)

In [ ]:
df.head()

In [ ]:
# df = pd.get_dummies(df, columns=['client_type'])

In [ ]:
df.shape

<a id="load_package"/><br/>
# TRAIN TEST SEPARATION

In [ ]:
# Separate train and test data from the combined dataframe
train = df[:ntrain]
test = df[ntrain:]

# Check the shapes of the split dataset
train.shape, test.shape

In [ ]:
train_df= train
test_df = test

<a id="load_package"/><br/>
## PAY ATTENTION TO THIS ATTRIBUTE

In [ ]:
train_df['loss_month_year'].value_counts().plot(kind='bar')

In [ ]:
train_df['loss_month_year'].value_counts()

In [ ]:
test_df['loss_month_year'].value_counts().plot(kind='bar')

We are going to use this field in our stratified kfold cross validation scheme since it has almost balanced points in both test and train

<a id="load_package"/><br/>
# FEATURE SELECTION

In [ ]:
# Select main columns to be used in training
main_cols = df.columns.difference(['claim_id','claim_number','policies_number','target'])
X = train_df[main_cols]
y = train_df.target
# Create the model with several hyperparameters
model = CatBoostRegressor(random_state = 42)
#
# Split into training and validation set
import random
random.seed(123)
#th
train_features, valid_features, train_y, valid_y = train_test_split(X, y, test_size = 0.2, random_state = 47,stratify=train_df["loss_month_year"])
# Train using early stopping
model.fit(train_features, train_y, 
         )

In [ ]:
pd.options.display.max_rows = 2200
#define a function to create variable importance dataframe
def get_lgbm_varimp(model, train_columns, max_vars=2000):
    
    # Scikit-learn API LGBMClassifier or LGBMRegressor was fitted, 
    # so using feature_importances_ property
    feature_importances_perc = (model.feature_importances_ / sum(model.feature_importances_)) * 100
    cv_varimp_df = pd.DataFrame([train_columns, feature_importances_perc]).T

    cv_varimp_df.columns = ['feature_name', 'varimp (%)']

    cv_varimp_df.sort_values(by='varimp (%)', ascending=False, inplace=True)

    cv_varimp_df = cv_varimp_df.iloc[0:max_vars]   

    return cv_varimp_df
#
get_lgbm_varimp(model, X.columns)

In [ ]:
#drop irrelevant columns/variables from X features
columns = [

]
#
X.drop(columns, inplace=True, axis=1)

In [ ]:
#Dropping highly correlated attributes 
corr = X.corr()
# sns.heatmap(corr)
#
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False
selected_columns = X.columns[columns]
X = X[selected_columns]

In [ ]:
train_df = train_df[selected_columns]
test_df = test_df[selected_columns]
train_df.shape,test_df.shape

In [ ]:
selected_columns

In [ ]:
# Separate train and test data from the combined dataframe
train = df[:ntrain]
test = df[ntrain:]

# Check the shapes of the split dataset
train.shape, test.shape

In [ ]:
train_df['target'] = train['target']
train_df['claim_id'] = train['claim_id']
train_df['loss_month_year'] = train['loss_month_year']
test_df['claim_id'] = test['claim_id']
test_df['loss_month_year'] = test['loss_month_year']

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df.to_csv(path+"/finaltrainimproved.csv", index=False)
test_df.to_csv(path+"/finaltestimproved.csv", index=False)

<a id="load_package"/><br/>
# MODELLING AND SUBMISSION

In [ ]:
from sklearn.model_selection import StratifiedKFold


In [ ]:
train = train_df
target = train.target.copy()
train.drop(['target','claim_id'], axis=1, inplace=True)
test = test_df
test.drop(['claim_id'], axis=1, inplace=True)

In [ ]:
train.shape,test.shape

In [ ]:
class func() :   
    def __init__(self, train, label, test, model, model_type, random_state):
        self.train, self.label, self.test = train, label, test
        self.model, self.model_type = model, model_type
        self.random_state = random_state
        
        assert self.model_type in ('catboost','xgboost'), 'Incorrect model_type'
    def __call__(self, plot = True):
        return self.fit(plot)

    def fit(self, plot):
        def catboost_fit(X_train, X_test, y_train, y_test):
            self.model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=500,
                           verbose=50,use_best_model=True)
            x_test_predict = self.model.predict(X_test)
            x_train_predict = self.model.predict(X_train)
            self.val_p[test_index] = x_test_predict
            self.test_p += self.model.predict(self.test)
            return x_test_predict, x_train_predict


        def xgb_fit(X_train, X_test, y_train, y_test):
            self.model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=500,
                           verbose=50,use_best_model=True)
            x_test_predict = self.model.predict(X_test)
            x_train_predict = self.model.predict(X_train)
            self.val_p[test_index] = x_test_predict
            self.test_p += self.model.predict(self.test)
            return x_test_predict, x_train_predict


        self.val_p = np.zeros(self.train.shape[0])
        mean_val = []
        mean_train = []
        self.test_p = np.zeros(self.test.shape[0])
        splits = 14
        kf = StratifiedKFold(n_splits = splits,shuffle = True, random_state=42)
        for fold_count, (train_index, test_index) in enumerate(kf.split(self.train, self.train['loss_month_year'])):
            X_train,X_test = self.train.iloc[train_index],self.train.iloc[test_index]
            y_train,y_test = self.label.iloc[train_index],self.label.iloc[test_index]

            print(f"================================Fold{fold_count+1}====================================")
            if self.model_type == 'catboost': x_test_predict, x_train_predict = catboost_fit(X_train, X_test, y_train, y_test)
            elif self.model_type == 'xgboost': x_test_predict, x_train_predict = xgb_fit(X_train, X_test, y_train, y_test)

            print('\nValidation scores', mean_absolute_error(y_test, x_test_predict), )
            print('Training scores', mean_absolute_error(y_train, x_train_predict),)
            mean_val.append(mean_absolute_error(y_test, x_test_predict))
            mean_train.append(mean_absolute_error(y_train, x_train_predict))

        if plot:
            feat_imp = pd.DataFrame(sorted(zip(self.model.feature_importances_,self.train.columns)), columns=['Value','Feature'])
            plt.figure(figsize=(30,25))
            sns.barplot(x="Value", y="Feature", data=feat_imp.sort_values(by="Value", ascending=False))
            plt.ylabel('Feature Importance Score')
            plt.show()
        print(np.mean(mean_val), np.mean(mean_train), np.std(mean_val))
        return self.val_p, self.test_p/splits, self.model

In [ ]:
catboost = CatBoostRegressor(n_estimators=10000, max_depth=6, eval_metric='MAE', reg_lambda = 370, )#task_type="GPU", devices='0:1'

func_= func(train, target, test, catboost, 'catboost', 16)
val_p1, test_p1, model1 = func_()

In [ ]:
catboost= CatBoostRegressor(bootstrap_type='Bernoulli', 
                              iterations=10000, silent=True, eval_metric='MAE')#,task_type="GPU", devices='0:1')
func_= func(train, target, test, catboost, 'catboost', 32)
val_p5, test_p5, model5 = func_()

In [ ]:
from sklearn.linear_model import  LinearRegression, Ridge, Lasso
stack = np.column_stack((val_p1,val_p5))

stack_p = np.column_stack((test_p1,test_p5))

predict = LinearRegression().fit(stack, target).predict(stack_p)

In [ ]:
submission['target'] = predict
submission.to_csv(path+ '/improved_catboost1.csv',index=False)

In [ ]:
submission.head()

In [ ]:
xgb_params1={'learning_rate': 0.03513119012633861, 'reg_lambda': 0.024374560039878566, 'reg_alpha': 217.32638594124083, 'subsample': 0.7682796533669832, 'colsample_bytree': 0.8491293374636897, 'max_depth': 6, 'random_state': 114, 'min_child_samples': 2, 'subsample_freq': 3}
xgb_params ={'learning_rate': 0.04402849326486322, 'reg_lambda': 0.00022496938162908366, 'reg_alpha': 0.0003614188253841325, 'subsample': 0.4276811873056746, 'colsample_bytree': 0.7670470835702196, 'max_depth': 5, 'min_child_samples': 1, 'subsample_freq': 7,'random_state':42}

In [ ]:
xgboost = XGBRegressor( n_jobs=4,
        **xgb_params1, eval_metric='mae',n_estimators=10000 )

func_= func(train, target, test, catboost, 'xgboost', 114)
val_p2, test_p2, model2 = func_()

In [ ]:
xgboost= XGBRegressor( n_jobs=4,
        **xgb_params,eval_metric='mae',n_estimators=7000)#,task_type="GPU", devices='0:1')
func_= func(train, target, test, catboost, 'xgboost', 42)
val_p3, test_p3, model3 = func_()

In [ ]:
from sklearn.linear_model import  LinearRegression, Ridge, Lasso
stack = np.column_stack((val_p2,val_p3))

stack_p = np.column_stack((test_p2,test_p3))

predict = LinearRegression().fit(stack, target).predict(stack_p)

In [ ]:
submission['target'] = predict
submission.to_csv(path+ '/improved_xgboost1.csv',index=False)

In [ ]:
from sklearn.linear_model import  LinearRegression, Ridge, Lasso
stack = np.column_stack((val_p2,val_p3,val_p1,val_p5))

stack_p = np.column_stack((test_p2,test_p3,test_p1,test_p5))

predict = LinearRegression().fit(stack, target).predict(stack_p)

In [ ]:
submission['target'] = predict
submission.to_csv(path+ '/improved_total1.csv',index=False)